In [1]:
import torch.nn as nn
from torchmetrics.functional import retrieval_normalized_dcg
from sklearn.metrics import ndcg_score
import numpy as np
import torch
import requests
from dataclasses import dataclass
from collections import defaultdict
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt

import sys
sys.path.append('BERT4Rec-VAE-Pytorch')

from models.bert import BERTModel
from models.bert_modules.bert import BERT

import warnings
warnings.filterwarnings("ignore")

### Service functions

In [23]:
def equalize_seq(seq, max_length, end_code):
  
  if len(seq) >= max_length:
    return seq[:max_length]
  else:
    return (max_length - len(seq)) * [end_code] + seq


def recall_k(y_pred, y_true, k=10):
    
    top_k = torch.topk(y_pred, k).indices.tolist()
    result = [el[0] in el[1] for el in zip(y_true.tolist(), top_k)]
    
    return np.mean(result)


def ndcg_k(y_pred, y_true, k=10):
    
    top_k = torch.topk(y_pred, k).indices.tolist()
    results = []
    for true_label, preds in zip(y_true.tolist(), top_k):
        tl = [[el == true_label for el in preds]]
        preds = [[1 for _ in preds]]
        results.append(ndcg_score(tl, preds))
    
    return np.mean(results)


def ndcg_k_seq(y_pred_seq, y_true_seq, k=10):
    recs = []
    for y_pred, y_true in zip(y_pred_seq, y_true_seq):
        recs.append(ndcg_k(y_pred, y_true, k))
    return np.mean(recs)


def recall_k_seq(y_pred_seq, y_true_seq, k=10):
    recs = []
    for y_pred, y_true in zip(y_pred_seq, y_true_seq):
        recs.append(recall_k(y_pred, y_true, k))
    return np.mean(recs)




def recall_k_seq_last(y_pred_seq, y_true_seq, k=10):
    recs = []
    for y_pred, y_true in zip(y_pred_seq, y_true_seq):
        recs.append(recall_k(y_pred, y_true, k))
    return recs[-1]


def ndcg_k_seq_last(y_pred_seq, y_true_seq, k=10):
    recs = []
    for y_pred, y_true in zip(y_pred_seq, y_true_seq):
        recs.append(ndcg_k(y_pred, y_true, k))
    return recs[-1]

In [3]:
device = "cuda:0"

### Load data

In [4]:
data = (pd.read_csv('ml-1m/ratings.dat', 
                    sep='::',
                    header=None,
                    index_col=0,
                    names=['user_id', 'movie_id', 'rating', 'ts'], 
                    engine='python')
          .reset_index(drop=False)
       )

sequences = data.sort_values(by=['user_id', 'ts']).groupby('user_id')['movie_id'].agg(lambda x: list(x)).to_dict()
sequences = {u: s for u, s in sequences.items() if len(s) > 0}

mask_code = 0
max_length = 100
end_code = data['movie_id'].max() + 1

### Train / Valid / Test Split

In [5]:
train = defaultdict(list)
val = {}
test = {}

for user, sequence in sequences.items():

    train[user] = equalize_seq(sequence[:-1], max_length=max_length, end_code=end_code)

    if np.random.choice([0, 1]):
        val[user] = equalize_seq(sequence, max_length=max_length, end_code=end_code)
      
    else:
        test[user] = equalize_seq(sequence, max_length=max_length, end_code=end_code)

test_indexes = np.array(test.keys())
val_indexes = np.array(val.keys())

### Define Model

In [6]:
@dataclass
class BertConf:
    bert_max_len: int
    num_items: int
    bert_num_blocks: int
    bert_num_heads: int
    bert_hidden_units: int
    bert_dropout: float = 0.1
    model_init_seed: int = 42

conf = BertConf(bert_max_len=max_length, 
                num_items=end_code + 1, 
                bert_num_blocks=2, 
                bert_num_heads=2, 
                bert_hidden_units=100)

model = BERTModel(conf).to(device)

In [7]:
import sys
sys.path.append('BERT4Rec-VAE-Pytorch')

from models.bert import BERTModel
from models.bert_modules.bert import BERT

from torch import nn as nn

from models.bert_modules.embedding import BERTEmbedding
from models.bert_modules.transformer import TransformerBlock
from utils import fix_random_seed_as
from models.base import BaseModel

import torch.nn as nn


class SasRec(nn.Module):
    def __init__(self, args):
        super().__init__()

        fix_random_seed_as(args.model_init_seed)

        max_len = args.bert_max_len
        num_items = args.num_items
        n_layers = args.bert_num_blocks
        heads = args.bert_num_heads
        vocab_size = num_items + 2
        hidden = args.bert_hidden_units
        self.hidden = hidden
        dropout = args.bert_dropout

        # embedding for BERT, sum of positional, segment, token embeddings
        self.embedding = BERTEmbedding(vocab_size=vocab_size, embed_size=self.hidden, max_len=max_len, dropout=dropout)

        # multi-layers transformer blocks, deep network
        self.transformer_blocks = nn.ModuleList(
            [TransformerBlock(hidden, heads, hidden * 4, dropout) for _ in range(n_layers)])

    def forward(self, x):
      
        # mask = (x > 0).unsqueeze(1).repeat(1, x.size(1), 1).unsqueeze(1)
        mask = torch.ones(x.size()[1], x.size()[1])
        mask = torch.tril(mask).unsqueeze(0).unsqueeze(1).repeat(x.size()[0], 1, 1, 1).to(device)

        # embedding the indexed sequence to sequence of vectors
        x = self.embedding(x.to(device))

        # running over multiple transformer blocks
        for transformer in self.transformer_blocks:
            x = transformer.forward(x, mask)

        return x

    def init_weights(self):
        pass


class SasRecModel(BaseModel):
    def __init__(self, args):
        super().__init__(args)
        self.sasrec = SasRec(args)
        self.out = nn.Linear(self.sasrec.hidden, args.num_items + 1)

    @classmethod
    def code(cls):
        return 'sasrec'

    def forward(self, x):
        x = self.sasrec(x)
        return self.out(x)

In [8]:
@dataclass
class SasRecConf:
    bert_max_len: int
    num_items: int
    bert_num_blocks: int
    bert_num_heads: int
    bert_hidden_units: int
    bert_dropout: float = 0.1
    model_init_seed: int = 42

conf = SasRecConf(bert_max_len=max_length, 
                  num_items=end_code + 1, 
                  bert_num_blocks=2, 
                  bert_num_heads=2, 
                  bert_hidden_units=100)

model = SasRecModel(conf).to(device)

### Define Dataloader

In [9]:
import torch
from torch.utils.data import Dataset, DataLoader

class TrainShiftedDataset(Dataset):
    def __init__(self, data, max_length, end_code=-1):
        self.data = data
        self.max_length = max_length
        self.end_code = end_code

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):

        sample = self.data[index]

        # Select random end of sequence
        idx = np.random.randint(len(sample))

        # Equalize sequence
        seq = equalize_seq(sample[:idx], self.max_length, self.end_code)
        seq = torch.tensor(seq)

        # Create X and y
        X = seq[:-1]
        y = seq[1:]
#         print(X.shape)
#         print(y.shape)

        return X, y
    

class ValidShiftedDataset(Dataset):
    def __init__(self, data, max_length, end_code=-1):
        self.data = data
        self.max_length = max_length
        self.end_code = end_code

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):

        sample = self.data[index]
        seq = equalize_seq(sample, self.max_length, self.end_code)
        seq = torch.tensor(seq)

        X = seq[:-1]
        y = seq[1:]

        return X, y

In [12]:
from tqdm import tqdm_notebook

### Train Model

In [14]:
dataset = TrainShiftedDataset(list(train.values()), max_length + 1, end_code)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=256, shuffle=True)

val_dataset = ValidShiftedDataset(list(val.values()), max_length + 1, end_code)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=256, shuffle=True)

# Using an Adam Optimizer with lr = 0.1
loss_function = torch.nn.CrossEntropyLoss(ignore_index=end_code, reduction='mean')
optimizer = torch.optim.Adam(model.parameters())

epochs = 5000
outputs = []
losses = []

counter = 0
recall_list = []
for epoch in range(epochs):
    for X, y in train_loader:

        optimizer.zero_grad()
        
        X = X.to(device)
        y = y.to(device)

        loss = loss_function(model.forward(X).view(-1, 3955), y.view(-1))

        losses.append(loss.item())
        loss.backward()
         
        optimizer.step()
        
        counter += 1
        if counter % 500 == 0:
            val_losses = []
            recall_batch = []
            ndcg_batch = []
            for X_val, y_val in tqdm_notebook(val_loader):
                
                X_val = X_val.to(device)
                y_val = y_val.to(device)
                
                y_pred = model.forward(X_val.long())
#                 print(y_pred.shape)
#                 print(y_val.shape)
                
                val_loss = loss_function(y_pred.view(-1, 3955), y_val.long().view(-1))
                val_losses.append(val_loss.item())
                
                recall_batch.append(recall_k_seq(y_pred, y_val, k=10))
#                 print("done")
#                 print(y_pred.shape)
#                 print(y_val.shape)
                ndcg_batch.append(ndcg_k_seq(y_pred, y_val))

#             print(counter, np.mean(recall_batch).round(3), np.round(loss.item(), 4), np.mean(val_losses).round(4))
            print("Epoch: ", counter)
            print("Recall@10: ", np.mean(recall_batch).round(3))
            print("NDCG: ", np.mean(ndcg_batch).round(3))
            print("Train Loss: ", np.round(loss.item(), 4))
            print("Val Loss: ", np.round(val_losses, 4))

  0%|          | 0/12 [00:00<?, ?it/s]

Epoch:  500
Recall@10:  0.169
NDCG:  0.077
Train Loss:  5.4897
Val Loss:  [5.6823 5.6948 5.7649 5.7182 5.7164 5.788  5.6832 5.6989 5.7924 5.724
 5.7121 5.6932]


  0%|          | 0/12 [00:00<?, ?it/s]

Epoch:  1000
Recall@10:  0.189
NDCG:  0.086
Train Loss:  5.3343
Val Loss:  [5.4845 5.4495 5.5367 5.587  5.5898 5.5735 5.4799 5.5981 5.5637 5.4716
 5.5179 5.6268]


  0%|          | 0/12 [00:00<?, ?it/s]

...

Epoch:  97500
Recall@10:  0.309
NDCG:  0.14
Train Loss:  3.9493
Val Loss:  [4.6355 4.6021 4.7205 4.7525 4.6455 4.7656 4.6584 4.7665 4.7243 4.7685
 4.6782 4.6413]


  0%|          | 0/12 [00:00<?, ?it/s]

Epoch:  98000
Recall@10:  0.308
NDCG:  0.14
Train Loss:  3.9751
Val Loss:  [4.7877 4.648  4.667  4.6569 4.7485 4.8098 4.735  4.7844 4.6634 4.6263
 4.6864 4.6346]


  0%|          | 0/12 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [25]:
import gc
gc.collect()

8586

In [26]:
test_dataset = ValidShiftedDataset(list(test.values()), max_length + 1, end_code)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=True)

recall_batch = []
ndcg_batch = []
recall_batch_last = []
recall_batch_last_n = []
ndcg_batch_last = []
for X_val, y_val in test_loader:

    X_val = X_val.to(device)
    y_val = y_val.to(device)

    y_pred = model.forward(X_val.long())

    val_loss = loss_function(y_pred.view(-1, 3955), y_val.long().view(-1))
    val_losses.append(val_loss.item())

    recall_batch.append(recall_k_seq(y_pred, y_val, k=1))
#                 print(y_pred.shape)
#                 print(y_val.shape)
    ndcg_batch.append(ndcg_k_seq(y_pred, y_val))
    recall_batch_last.append(recall_k_seq_last(y_pred, y_val, k=1))
    recall_batch_last_n.append(recall_k_seq_last(y_pred, y_val, k=10))
    ndcg_batch_last.append(ndcg_k_seq_last(y_pred, y_val))

#             print(counter, np.mean(recall_batch).round(3), np.round(loss.item(), 4), np.mean(val_losses).round(4))
print("Epoch: ", counter)
print("Recall@10: ", np.mean(recall_batch).round(3))
print("NDCG: ", np.mean(ndcg_batch).round(3))
print("Train Loss: ", np.round(loss.item(), 4))
print("Val Loss: ", np.round(val_losses, 4))

Epoch:  98500
Recall@10:  0.092
NDCG:  0.138
Train Loss:  3.92
Val Loss:  [4.6728 4.6405 4.7094 4.798  4.7703 4.6332 4.7308 4.8353 4.6827 4.8238
 4.2946 4.6055 4.6474 4.632  4.7355 4.7523 4.5849 4.5989 4.8591 4.7677
 4.7396 4.6966 4.6024 4.6339 4.6872 4.5297 4.8695 4.6202 4.7405 4.6792
 4.6016 4.8307 4.9578 4.7885 4.5176 4.5662 4.3075 4.8498 4.6536 4.9235
 4.6923 4.8991 4.8624 4.8197 4.6336 4.7685 4.6627 4.9122 5.0828 4.6217
 4.6903 4.6583 4.5473 4.7504 4.8575 4.9014 4.4955 4.9811 4.8286 4.4444
 4.584  4.7305 4.5577 4.5488 4.589  4.651  4.8433 4.7886 4.5364 5.0141
 4.7882 4.5102 4.6859 4.7054 4.7765 4.6968 4.4849 4.8468 4.666  4.5735
 4.5979 4.7585 4.6788 4.8862 4.6489 4.6163 4.6726 4.7143 4.843  4.8543
 4.5983 4.6586 4.7392 4.787  4.764  4.6393 4.7162 4.7799 4.7643 4.6617
 4.6654 4.5783 4.7894 4.7173 4.1557 4.7219 4.8763 4.6991 4.7657 4.6006
 4.6485 4.4606 4.6346 4.7261 4.5867 4.6432 4.643  4.7125 4.7216 4.8548
 4.743  4.6442 4.5904 4.7911 4.8246 4.7503 4.626  4.5921 4.5727 4.5728
 4.

In [27]:
print("Epoch: ", counter)
print("Recall@10: ", np.mean(recall_batch_last_n).round(3))
print("Recall@1: ", np.mean(recall_batch_last).round(3))
print("NDCG: ", np.mean(ndcg_batch_last).round(3))


Epoch:  98500
Recall@10:  0.294
Recall@1:  0.086
NDCG:  0.134


In [ ]:
Epoch:  98500
Recall@10:  0.305
NDCG:  0.138

In [ ]:
epochs = 15000
for epoch in range(epochs):
    for X, y in train_loader:

        optimizer.zero_grad()
        
        X = X.to('cuda:0')
        y = y.to('cuda:0')

        loss = loss_function(model.forward(X).view(-1, 3955), y.view(-1))

        losses.append(loss.item())
        loss.backward()
         
        optimizer.step()
        
        counter += 1
        if counter % 500 == 0:
            val_losses = []
            recall_batch = []
            ndcg_batch = []
            for X_val, y_val in val_loader:
                
                X_val = X_val.to('cuda:0')
                y_val = y_val.to('cuda:0')
                
                y_pred = model.forward(X_val.long())[:, -1, :]
                
                val_loss = loss_function(y_pred.view(-1, 3955), y_val.long().view(-1))
                val_losses.append(val_loss.item())
                
                recall_batch.append(recall_k(y_pred, y_val, k=10))
#                 print(y_pred.shape)
#                 print(y_val.shape)
                ndcg_batch.append(ndcg_k(y_pred, y_val))

#             print(counter, np.mean(recall_batch).round(3), np.round(loss.item(), 4), np.mean(val_losses).round(4))
            print("Epoch: ", counter)
            print("Recall@10: ", np.mean(recall_batch).round(3))
            print("NDCG: ", np.mean(ndcg_batch).round(3))
            print("Train Loss: ", np.round(loss.item(), 4))
            print("Val Loss: ", np.round(val_losses, 4))